In [1]:
import os
import numpy as np
import pandas as pd
%matplotlib qt
import matplotlib.pyplot as plt
import seaborn as sns
from skimage import io
from skimage import measure
from scipy import ndimage as ndi
from skimage.morphology import binary_dilation, disk


In [2]:
segm_img_dir = r'Z:\TAD\200428_adipogenesis_timecourses_revisited\190718_3t3l1_time_course_r5\filtered_segm_imgs' #directory of segmented images
cebp_segm_dir = r'Z:\TAD\200428_adipogenesis_timecourses_revisited\190718_3t3l1_time_course_r5\cebp_segmented_imgs'
dapi_segm_dir = r'Z:\TAD\200428_adipogenesis_timecourses_revisited\190718_3t3l1_time_course_r5\dapi_segmented_imgs'
orig_img_dir_base = r'Z:\TAD\190718_3t3l1_time_course_r5' #directory that has subdirectories with original images 

In [3]:
os.chdir(cebp_segm_dir)


In [8]:
base_name = os.listdir()[0][:-20]
base_name

'stad3-26-xy1'

In [4]:
os.chdir(segm_img_dir)

img_lst_temp = []

for filename in os.listdir():
    if '.tif' in filename:
        img_lst_temp.append(filename[:-20])
img_lst = np.unique(img_lst_temp)
img_lst

array(['stad3-26', 'stad3-27', 'stad3-28', 'stad3-29', 'stad3-30',
       'stad3-31', 'stad3-32', 'stad3-33', 'stad3-34', 'stad3-35',
       'stad3-36', 'stad3-37', 'stad3-38', 'stad3-39', 'stad3-40',
       'stad3-41', 'stad3-42', 'stad3-43', 'stad3-44', 'stad3-45',
       'stad3-46', 'stad3-47', 'stad3-48'], dtype='<U8')

In [5]:
'''generate a dictionary that has associated times for each image'''
os.chdir(segm_img_dir)

img_lst_temp = []

for filename in os.listdir():
    if '.tif' in filename:
        img_lst_temp.append(filename[:-20])
img_lst_4_times = np.unique(img_lst_temp)


times_4_dict = [-3, -1, 0, 0, .083, .083, .167, .167, .25, .25, .33, .33, 1, 1, 2, 2, 3, 3, 4, 4, 6, 6, -1] #list of times associated with images
times_dict = {}

for i in range(len(img_lst_4_times)):
    img_name = img_lst_4_times[i]
    time = times_4_dict[i]
    
    times_dict.update({img_name : time})

In [82]:
'''this is a helper function to invert a binary mask. It takes in a binary mask and makes a negative'''
def invert_mask(masked_img_):
    '''just to make sure that everything is in the right form I am running quick bit to guarantee it is 1s and 0s'''
    masked_img_max = np.max(masked_img_) 
    masked_img_binary = np.divide(masked_img_, masked_img_max) #changes to 1s and 0s
    ones_bool = np.ones([masked_img_.shape[0], masked_img_.shape[1]])
    return(np.subtract(ones_bool, masked_img_binary))

In [161]:
'''series of lists to be populated with values in the script'''

segm_cebp_imagename_global_lst = [] #segmeneted cebp image filename

day_global_lst = [] #associated time with each image


'''total nuclear intensities'''
area_nucleus_global_lst = [] #area of individual nucleus
xcoord_nucleus_global_lst = []
ycoord_nucleus_global_lst = []
dapi_nucleus_global_lst = [] #dapi in nuclues
cebp_nuclues_global_lst = [] #cebp in nucleus
ppar_nuclues_global_lst = [] #ppar in nucleus
'''heterochromatin intensities'''
area_heteroc_global_lst = []
dapi_heteroc_global_lst = []
cebp_heteroc_global_lst = []
ppar_heteroc_global_lst = []
'''!heterochromatin intens'''
area_n_heteroc_global_lst = []
dapi_n_heteroc_global_lst = []
cebp_n_heteroc_global_lst = []
ppar_n_heteroc_global_lst = []
'''heterochromatin +3 ring intensities'''
area_p3_heteroc_global_lst = []
dapi_p3_heteroc_global_lst = []
cebp_p3_heteroc_global_lst = []
ppar_p3_heteroc_global_lst = []
'''~heterochromatin +3 ring intensities'''
area_n_p3_heteroc_global_lst = []
dapi_n_p3_heteroc_global_lst = []
cebp_n_p3_heteroc_global_lst = []
ppar_n_p3_heteroc_global_lst = []
'''cebp blob intensities'''
area_blob_global_lst = []
dapi_blob_global_lst = []
cebp_blob_global_lst = []
ppar_blob_global_lst = []
'''~cebp blob intensiities'''
area_n_blob_global_lst = []
dapi_n_blob_global_lst = []
cebp_n_blob_global_lst = []
ppar_n_blob_global_lst = []
'''cebp blob +1ring intensties'''
area_p1_blob_global_lst = []
dapi_p1_blob_global_lst = []
cebp_p1_blob_global_lst = []
ppar_p1_blob_global_lst = []
'''~cebp blob +1ring intensties'''
area_n_p1_blob_global_lst = []
dapi_n_p1_blob_global_lst = []
cebp_n_p1_blob_global_lst = []
ppar_n_p1_blob_global_lst = []
'''cebp blob + 2 ring intensties'''
area_p2_blob_global_lst = []
dapi_p2_blob_global_lst = []
cebp_p2_blob_global_lst = []
ppar_p2_blob_global_lst = []
'''!cebp blob + 2 ring intensties'''
area_n_p2_blob_global_lst = []
dapi_n_p2_blob_global_lst = []
cebp_n_p2_blob_global_lst = []
ppar_n_p2_blob_global_lst = []
'''cebp blob + 3 ring intensties'''
area_p3_blob_global_lst = []
dapi_p3_blob_global_lst = []
cebp_p3_blob_global_lst = []
ppar_p3_blob_global_lst = []
'''!cebp blob + 3 ring intensties'''
area_n_p3_blob_global_lst = []
dapi_n_p3_blob_global_lst = []
cebp_n_p3_blob_global_lst = []
ppar_n_p3_blob_global_lst = []
'''cebp blob + 5 ring intensties'''
area_p5_blob_global_lst = []
dapi_p5_blob_global_lst = []
cebp_p5_blob_global_lst = []
ppar_p5_blob_global_lst = []
'''!cebp blob + 5 ring intensties'''
area_n_p5_blob_global_lst = []
dapi_n_p5_blob_global_lst = []
cebp_n_p5_blob_global_lst = []
ppar_n_p5_blob_global_lst = []
'''cebp blob + 7 ring intensties'''
area_p7_blob_global_lst = []
dapi_p7_blob_global_lst = []
cebp_p7_blob_global_lst = []
ppar_p7_blob_global_lst = []
'''!cebp blob + 7 ring intensties'''
area_n_p7_blob_global_lst = []
dapi_n_p7_blob_global_lst = []
cebp_n_p7_blob_global_lst = []
ppar_n_p7_blob_global_lst = []

os.chdir(cebp_segm_dir)

'''pull all of the .tif files out of the directory and put them into a list'''
cebp_segm_img_lst = []
for filename in os.listdir():
    if '.tif' in filename:
        cebp_segm_img_lst.append(filename)

        
filename_base = 'this is for a little later in code to check if files are already read into memory'

#########################################################
cebp_segm_img_lst = cebp_segm_img_lst[-14 : -8] #this is temporary while working on code
################################################

for segm_cebp_img_filename in cebp_segm_img_lst:
    os.chdir(cebp_segm_dir)
    segm_cebp_imagename_global_lst.append(segm_cebp_img_filename) #add filename to running list for bookkeeping
   
    
    cebp_segm_img = io.imread(segm_cebp_img_filename) #read in cebp single cell segmented image
    
    
    os.chdir(dapi_segm_dir) #switch over to segmented dapi directory and read in segmented dapi image'''
    
    '''some images have more than 10 images which throws off the naming conventions. This should handle this'''
    if len(segm_cebp_img_filename) == 32:
        segm_dapi_img_filename = segm_cebp_img_filename[:-19]+'dapi'+segm_cebp_img_filename[-15:]
        file_name_test = segm_cebp_img_filename[:-20]
    if len(segm_cebp_img_filename) == 33:
        segm_dapi_img_filename = segm_cebp_img_filename[:-20]+'dapi'+segm_cebp_img_filename[-16:]
        file_name_test = segm_cebp_img_filename[:-21]
    dapi_segm_img = io.imread(segm_dapi_img_filename)
    
    
    '''the segmented nuclei image and the fluorescent images may already be in memory, in which case can save time by just leaving them
    there and not reading them in over and over'''
  

    if filename_base != file_name_test:

        '''read in segmented nuclei and fluorescent images'''
        filename_base = file_name_test
        os.chdir(segm_img_dir) #switch over to the segmented nuceli image directory and pull out the individual nuclei'''
        nuclei_segm_img = io.imread(filename_base+'segm_parsed.tiff') #pull out the base filename in order to identify other images associated with this mask'''
        os.chdir(orig_img_dir_base+'\\'+segm_cebp_img_filename[:-24]) #switch over to the directory with the original fluoresence images and read in dapi, cebp and ppar images'''
        dapi_orig_img = io.imread(filename_base+'c1.tif') #read in original dapi image
        ppar_orig_img = io.imread(filename_base+'c3.tif') #read in original ppar image
        cebp_orig_img = io.imread(filename_base+'c4.tif') #read in original cebp image
    
    '''pull out the individual nuclei from this image that corresponds to the segmented cebp and dapi images'''
    labeled_nuclei_segm_img = measure.label(nuclei_segm_img) #label the objects within the image 
    nuclei_number = int(segm_cebp_img_filename[-5]) #find the associated nucleus in the image based on the filename

    nucleus_segm_img = np.array(labeled_nuclei_segm_img == nuclei_number).astype('uint16') #pull out the single nuclues
    
    '''find the associated day for the image being analyzed'''
    day = times_dict[filename_base[:-4]] #pull appropriate time out of dictionary based on the filename
    day_global_lst.append(day)
    '''now the fun begins'''
    area_nucleus_global_lst.append(np.sum(nucleus_segm_img)) #determine total area of the nucleus
    xcoord_nucleus_global_lst.append(measure.regionprops(nucleus_segm_img)[0].centroid[1]) #determine x coordinate for center of mass
    ycoord_nucleus_global_lst.append(measure.regionprops(nucleus_segm_img)[0].centroid[0]) #determine y coordinate for center of mass
    '''determine the integrated intesity in the nucleus for each fluorescent channel'''
    dapi_nucleus_global_lst.append(np.sum(np.multiply(nucleus_segm_img, dapi_orig_img))) #dapi in nuclues
    cebp_nuclues_global_lst.append(np.sum(np.multiply(nucleus_segm_img, cebp_orig_img))) #cebp in nucleus
    ppar_nuclues_global_lst.append(np.sum(np.multiply(nucleus_segm_img, ppar_orig_img))) #ppar in nucleus
    '''determine the integrated intensity in the dapi mask'''
    area_heteroc_global_lst.append(np.sum(dapi_segm_img))
    dapi_heteroc_global_lst.append(np.sum(np.multiply(dapi_segm_img, dapi_orig_img)))
    cebp_heteroc_global_lst.append(np.sum(np.multiply(dapi_segm_img, cebp_orig_img)))
    ppar_heteroc_global_lst.append(np.sum(np.multiply(dapi_segm_img, ppar_orig_img)))
    '''determin the integrated intensity outside of the dapi mask but in the nucleus'''
    n_heteroc_mask = np.multiply(nucleus_segm_img, invert_mask(dapi_segm_img)) #mask of region in the nucleus outside of the dapi mask
    area_n_heteroc_global_lst.append(np.sum(n_heteroc_mask))
    dapi_n_heteroc_global_lst.append(np.sum(np.multiply(n_heteroc_mask, dapi_orig_img)))
    cebp_n_heteroc_global_lst.append(np.sum(np.multiply(n_heteroc_mask, cebp_orig_img)))
    ppar_n_heteroc_global_lst.append(np.sum(np.multiply(n_heteroc_mask, ppar_orig_img)))
    '''determine integrated intensity around the periphery of the dapi mask'''
    heteroc_dilate_p3 = binary_dilation(dapi_segm_img, disk(3)).astype('uint16')
    p3_heteroc_mask = np.subtract(heteroc_dilate_p3, dapi_segm_img)
    area_p3_heteroc_global_lst.append(np.sum(p3_heteroc_mask))
    dapi_p3_heteroc_global_lst.append(np.sum(np.multiply(p3_heteroc_mask, dapi_orig_img)))
    cebp_p3_heteroc_global_lst.append(np.sum(np.multiply(p3_heteroc_mask, cebp_orig_img)))
    ppar_p3_heteroc_global_lst.append(np.sum(np.multiply(p3_heteroc_mask, ppar_orig_img)))
    '''determine integrated intensity in nucleus but not in heteroc or p3_heteroc_mask'''
    n_p3_heteroc_mask = np.multiply(nucleus_segm_img, invert_mask(heteroc_dilate_p3)) #mask of region in the nucleus outside of the dapi mask
    area_n_p3_heteroc_global_lst.append(np.sum(n_p3_heteroc_mask))
    dapi_n_p3_heteroc_global_lst.append(np.sum(np.multiply(n_p3_heteroc_mask, dapi_orig_img)))
    cebp_n_p3_heteroc_global_lst.append(np.sum(np.multiply(n_p3_heteroc_mask, cebp_orig_img)))
    ppar_n_p3_heteroc_global_lst.append(np.sum(np.multiply(n_p3_heteroc_mask, ppar_orig_img)))
    '''determine integrated intensity in the cebp blobs'''
    area_blob_global_lst.append(np.sum(cebp_segm_img))
    dapi_blob_global_lst.append(np.sum(np.multiply(cebp_segm_img, dapi_orig_img)))
    cebp_blob_global_lst.append(np.sum(np.multiply(cebp_segm_img, cebp_orig_img)))
    ppar_blob_global_lst.append(np.sum(np.multiply(cebp_segm_img, ppar_orig_img)))
    '''determine integrated intensity outside of teh cebp blobs but in the nucleus'''
    n_blob_mask = np.multiply(nucleus_segm_img, invert_mask(cebp_segm_img))
    area_n_blob_global_lst.append(np.sum(n_blob_mask))
    dapi_n_blob_global_lst.append(np.sum(np.multiply(n_blob_mask, dapi_orig_img)))
    cebp_n_blob_global_lst.append(np.sum(np.multiply(n_blob_mask, cebp_orig_img)))
    ppar_n_blob_global_lst.append(np.sum(np.multiply(n_blob_mask, ppar_orig_img)))
    '''determine integrated intensity around +1 the periphery of cebp blobs'''
    blob_dilate_p1 = binary_dilation(cebp_segm_img, disk(1)).astype('uint16')
    p1_blob_mask = np.subtract(blob_dilate_p1, cebp_segm_img)
    area_p1_blob_global_lst.append(np.sum(p1_blob_mask))
    dapi_p1_blob_global_lst.append(np.sum(np.multiply(p1_blob_mask, dapi_orig_img)))
    cebp_p1_blob_global_lst.append(np.sum(np.multiply(p1_blob_mask, cebp_orig_img)))
    ppar_p1_blob_global_lst.append(np.sum(np.multiply(p1_blob_mask, ppar_orig_img)))
    '''determine integrated intensity in nucleus but not around the +1 periphery of the cebp blobs'''
    n_p1_blob_mask = np.multiply(nucleus_segm_img, invert_mask(blob_dilate_p1))
    area_n_p1_blob_global_lst.append(np.sum(n_p1_blob_mask))
    dapi_n_p1_blob_global_lst.append(np.sum(np.multiply(n_p1_blob_mask, dapi_orig_img)))
    cebp_n_p1_blob_global_lst.append(np.sum(np.multiply(n_p1_blob_mask, cebp_orig_img)))
    ppar_n_p1_blob_global_lst.append(np.sum(np.multiply(n_p1_blob_mask, ppar_orig_img)))
    '''determine integrated intensity around +2 the periphery of cebp blobs'''
    blob_dilate_p2 = binary_dilation(cebp_segm_img, disk(2)).astype('uint16')
    p2_blob_mask = np.subtract(blob_dilate_p2, blob_dilate_p1)
    area_p2_blob_global_lst.append(np.sum(p2_blob_mask))
    dapi_p2_blob_global_lst.append(np.sum(np.multiply(p2_blob_mask, dapi_orig_img)))
    cebp_p2_blob_global_lst.append(np.sum(np.multiply(p2_blob_mask, cebp_orig_img)))
    ppar_p2_blob_global_lst.append(np.sum(np.multiply(p2_blob_mask, ppar_orig_img)))
    '''determine integrated intensity in nucleus but not around the +2 periphery of the cebp blobs'''
    n_p2_blob_mask = np.multiply(nucleus_segm_img, invert_mask(blob_dilate_p2))
    area_n_p2_blob_global_lst.append(np.sum(n_p2_blob_mask))
    dapi_n_p2_blob_global_lst.append(np.sum(np.multiply(n_p2_blob_mask, dapi_orig_img)))
    cebp_n_p2_blob_global_lst.append(np.sum(np.multiply(n_p2_blob_mask, cebp_orig_img)))
    ppar_n_p2_blob_global_lst.append(np.sum(np.multiply(n_p2_blob_mask, ppar_orig_img)))
    '''determine integrated intensity around +3 the periphery of cebp blobs'''
    blob_dilate_p3 = binary_dilation(cebp_segm_img, disk(3)).astype('uint16')
    p3_blob_mask = np.subtract(blob_dilate_p3, blob_dilate_p2)
    area_p3_blob_global_lst.append(np.sum(p3_blob_mask))
    dapi_p3_blob_global_lst.append(np.sum(np.multiply(p3_blob_mask, dapi_orig_img)))
    cebp_p3_blob_global_lst.append(np.sum(np.multiply(p3_blob_mask, cebp_orig_img)))
    ppar_p3_blob_global_lst.append(np.sum(np.multiply(p3_blob_mask, ppar_orig_img)))
    '''determine integrated intensity in nucleus but not around the +3 periphery of the cebp blobs'''
    n_p3_blob_mask = np.multiply(nucleus_segm_img, invert_mask(blob_dilate_p3))
    area_n_p3_blob_global_lst.append(np.sum(n_p3_blob_mask))
    dapi_n_p3_blob_global_lst.append(np.sum(np.multiply(n_p3_blob_mask, dapi_orig_img)))
    cebp_n_p3_blob_global_lst.append(np.sum(np.multiply(n_p3_blob_mask, cebp_orig_img)))
    ppar_n_p3_blob_global_lst.append(np.sum(np.multiply(n_p3_blob_mask, ppar_orig_img)))
    '''determine integrated intensity around +5 the periphery of cebp blobs'''
    blob_dilate_p5 = binary_dilation(cebp_segm_img, disk(5)).astype('uint16')
    p5_blob_mask = np.subtract(blob_dilate_p5, blob_dilate_p3)
    area_p5_blob_global_lst.append(np.sum(p5_blob_mask))
    dapi_p5_blob_global_lst.append(np.sum(np.multiply(p5_blob_mask, dapi_orig_img)))
    cebp_p5_blob_global_lst.append(np.sum(np.multiply(p5_blob_mask, cebp_orig_img)))
    ppar_p5_blob_global_lst.append(np.sum(np.multiply(p5_blob_mask, ppar_orig_img)))
    '''determine integrated intensity in nucleus but not around the +5 periphery of the cebp blobs'''
    n_p5_blob_mask = np.multiply(nucleus_segm_img, invert_mask(blob_dilate_p5))
    area_n_p5_blob_global_lst.append(np.sum(n_p5_blob_mask))
    dapi_n_p5_blob_global_lst.append(np.sum(np.multiply(n_p5_blob_mask, dapi_orig_img)))
    cebp_n_p5_blob_global_lst.append(np.sum(np.multiply(n_p5_blob_mask, cebp_orig_img)))
    ppar_n_p5_blob_global_lst.append(np.sum(np.multiply(n_p5_blob_mask, ppar_orig_img)))
    '''determine integrated intensity around +3 the periphery of cebp blobs'''
    blob_dilate_p7 = binary_dilation(cebp_segm_img, disk(7)).astype('uint16')
    p7_blob_mask = np.subtract(blob_dilate_p7, blob_dilate_p5)
    area_p7_blob_global_lst.append(np.sum(p7_blob_mask))
    dapi_p7_blob_global_lst.append(np.sum(np.multiply(p7_blob_mask, dapi_orig_img)))
    cebp_p7_blob_global_lst.append(np.sum(np.multiply(p7_blob_mask, cebp_orig_img)))
    ppar_p7_blob_global_lst.append(np.sum(np.multiply(p7_blob_mask, ppar_orig_img)))
    '''determine integrated intensity in nucleus but not around the +2 periphery of the cebp blobs'''
    n_p7_blob_mask = np.multiply(nucleus_segm_img, invert_mask(blob_dilate_p7))
    area_n_p7_blob_global_lst.append(np.sum(n_p7_blob_mask))
    dapi_n_p7_blob_global_lst.append(np.sum(np.multiply(n_p7_blob_mask, dapi_orig_img)))
    cebp_n_p7_blob_global_lst.append(np.sum(np.multiply(n_p7_blob_mask, cebp_orig_img)))
    ppar_n_p7_blob_global_lst.append(np.sum(np.multiply(n_p7_blob_mask, ppar_orig_img)))
    
    

C:\Users\LegantLab\anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [158]:
segm_cebp_img_filename[:-24]

'stad3-44-'

In [160]:
filename_base[:-4]

'stad3-44'

In [151]:
filename_base[:-1]

'stad3-44-xy5'

In [139]:
cebp_segm_img_lst

['stad3-43-xy5_cebp_segm_cell7.tif',
 'stad3-43-xy5_cebp_segm_cell8.tif',
 'stad3-43-xy5_cebp_segm_cell9.tif',
 'stad3-44-xy5_cebp_segm_cell1.tif',
 'stad3-44-xy5_cebp_segm_cell10.tif',
 'stad3-44-xy5_cebp_segm_cell11.tif']

In [132]:
segm_cebp_img_filename[:-20]+'dapi'+segm_cebp_img_filename[-16:]

'stad3-44-xy5_dapi_segm_cell10.tif'

In [129]:
segm_dapi_img_filename

'stad3-44-xy5_cdapisegm_cell10.tif'

In [128]:
len(segm_dapi_img_filename)

33

In [162]:
'''this is testing out to make sure can throw everything into a dataframe. make sure that lists are the same shape etc. '''
df_ = pd.DataFrame({'segm_cebp_imagename' :segm_cebp_imagename_global_lst,                   
                    'day' : day_global_lst,
                    'area_nucleus' : area_nucleus_global_lst,
                    'xcoord_nucleus' : xcoord_nucleus_global_lst,
                    'ycoord_nucleus' : ycoord_nucleus_global_lst,
                    'dapi_nucleus' : dapi_nucleus_global_lst,
                    'cebp_nuclues' : cebp_nuclues_global_lst,
                    'ppar_nuclues' : ppar_nuclues_global_lst,
                    'area_heteroc' : area_heteroc_global_lst,
                    'dapi_heteroc' : dapi_heteroc_global_lst,
                    'cebp_heteroc' : cebp_heteroc_global_lst,
                    'ppar_heteroc' : ppar_heteroc_global_lst,
                    'area_n_heteroc' : area_n_heteroc_global_lst,
                    'dapi_n_heteroc' : dapi_n_heteroc_global_lst,
                    'cebp_n_heteroc' : cebp_n_heteroc_global_lst,
                    'ppar_n_heteroc' : ppar_n_heteroc_global_lst,
                    'area_p3_heteroc' : area_p3_heteroc_global_lst,
                    'dapi_p3_heteroc' : dapi_p3_heteroc_global_lst,
                    'cebp_p3_heteroc' : cebp_p3_heteroc_global_lst,
                    'ppar_p3_heteroc' : ppar_p3_heteroc_global_lst,
                    'area_n_p3_heteroc' : area_n_p3_heteroc_global_lst,
                    'dapi_n_p3_heteroc' : dapi_n_p3_heteroc_global_lst,
                    'cebp_n_p3_heteroc' : cebp_n_p3_heteroc_global_lst,
                    'ppar_n_p3_heteroc' : ppar_n_p3_heteroc_global_lst,
                    'area_blob' : area_blob_global_lst,
                    'dapi_blob' : dapi_blob_global_lst,
                    'cebp_blob' : cebp_blob_global_lst,
                    'ppar_blob' : ppar_blob_global_lst,
                    'area_n_blob' : area_n_blob_global_lst,
                    'dapi_n_blob' : dapi_n_blob_global_lst,
                    'cebp_n_blob' : cebp_n_blob_global_lst,
                    'ppar_n_blob' : ppar_n_blob_global_lst,
                    'area_p1_blob' : area_p1_blob_global_lst,
                    'dapi_p1_blob' : dapi_p1_blob_global_lst,
                    'cebp_p1_blob' : cebp_p1_blob_global_lst,
                    'ppar_p1_blob' : ppar_p1_blob_global_lst,
                    'area_n_p1_blob' : area_n_p1_blob_global_lst,
                    'dapi_n_p1_blob' : dapi_n_p1_blob_global_lst,
                    'cebp_n_p1_blob' : cebp_n_p1_blob_global_lst,
                    'ppar_n_p1_blob' : ppar_n_p1_blob_global_lst,
                    'area_p2_blob' : area_p2_blob_global_lst,
                    'dapi_p2_blob' : dapi_p2_blob_global_lst,
                    'cebp_p2_blob' : cebp_p2_blob_global_lst,
                    'ppar_p2_blob' : ppar_p2_blob_global_lst,
                    'area_n_p2_blob' : area_n_p2_blob_global_lst,
                    'dapi_n_p2_blob' : dapi_n_p2_blob_global_lst,
                    'cebp_n_p2_blob' : cebp_n_p2_blob_global_lst,
                    'ppar_n_p2_blob' : ppar_n_p2_blob_global_lst,
                    'area_p3_blob' : area_p3_blob_global_lst,
                    'dapi_p3_blob' : dapi_p3_blob_global_lst,
                    'cebp_p3_blob' : cebp_p3_blob_global_lst,
                    'ppar_p3_blob' : ppar_p3_blob_global_lst,
                    'area_n_p3_blob' : area_n_p3_blob_global_lst,
                    'dapi_n_p3_blob' : dapi_n_p3_blob_global_lst,
                    'cebp_n_p3_blob' : cebp_n_p3_blob_global_lst,
                    'ppar_n_p3_blob' : ppar_n_p3_blob_global_lst,
                    'area_p5_blob' : area_p5_blob_global_lst,
                    'dapi_p5_blob' : dapi_p5_blob_global_lst,
                    'cebp_p5_blob' : cebp_p5_blob_global_lst,
                    'ppar_p5_blob' : ppar_p5_blob_global_lst,
                    'area_n_p5_blob' : area_n_p5_blob_global_lst,
                    'dapi_n_p5_blob' : dapi_n_p5_blob_global_lst,
                    'cebp_n_p5_blob' : cebp_n_p5_blob_global_lst,
                    'ppar_n_p5_blob' : ppar_n_p5_blob_global_lst,
                    'area_p7_blob' : area_p7_blob_global_lst,
                    'dapi_p7_blob' : dapi_p7_blob_global_lst,
                    'cebp_p7_blob' : cebp_p7_blob_global_lst,
                    'ppar_p7_blob' : ppar_p7_blob_global_lst,
                    'area_n_p7_blob' : area_n_p7_blob_global_lst,
                    'dapi_n_p7_blob' : dapi_n_p7_blob_global_lst,
                    'cebp_n_p7_blob' : cebp_n_p7_blob_global_lst,
                    'ppar_n_p7_blob' : ppar_n_p7_blob_global_lst
                   })
                    


In [163]:
df_

,segm_cebp_imagename,day,area_nucleus,xcoord_nucleus,ycoord_nucleus,dapi_nucleus,cebp_nuclues,ppar_nuclues,area_heteroc,dapi_heteroc,...,cebp_n_p5_blob,ppar_n_p5_blob,area_p7_blob,dapi_p7_blob,cebp_p7_blob,ppar_p7_blob,area_n_p7_blob,dapi_n_p7_blob,cebp_n_p7_blob,ppar_n_p7_blob
0,stad3-43-xy5_cebp_segm_cell7.tif,3,25191,1084.516256,909.431980,130612043,133104142,148222283,683,6313105,...,8.705753e+07,1.102179e+08,1245,8370753,9202661,10236977,19748.0,9.439920e+07,7.785487e+07,9.998089e+07
1,stad3-43-xy5_cebp_segm_cell8.tif,3,37376,1549.234964,1013.309156,140667642,130945978,86938453,0,0,...,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN
2,stad3-43-xy5_cebp_segm_cell9.tif,3,40073,1333.199835,1026.168043,138542107,108306718,84923265,911,4017442,...,7.988894e+07,6.598274e+07,2628,9462574,7751623,5740737,28875.0,9.819429e+07,7.217948e+07,6.027923e+07
3,stad3-44-xy5_cebp_segm_cell1.tif,4,25960,475.551079,158.588906,160114300,107006098,92094120,1080,8442602,...,9.354160e+07,8.249196e+07,896,6191705,4405230,3436337,22581.0,1.366848e+08,8.913637e+07,7.905562e+07
4,stad3-44-xy5_cebp_segm_cell10.tif,4,3834810,1019.739775,1031.293275,1625024523,3152704121,1704884759,2239,27905765,...,7.447671e+09,1.029482e+10,2729,25865684,20594119,20930837,3834807.0,1.450991e+10,7.447664e+09,1.029481e+10
5,stad3-44-xy5_cebp_segm_cell11.tif,4,25960,475.551079,158.588906,160114300,107006098,92094120,1159,12083173,...,1.070061e+08,9.209412e+07,2262,17195582,9835779,13823482,25960.0,1.601143e+08,1.070061e+08,9.209412e+07


In [101]:
minr, minc, maxr, maxc = measure.regionprops(nucleus_segm_img)[0].bbox #determine a bounding box for the image

fig, axes = plt.subplots(5, 4)
axes = axes.flatten()
axes[0].imshow(np.array(nucleus_segm_img)[minr:maxr, minc:maxc])
axes[0].title.set_text('segmented nucleus')
axes[1].imshow((np.multiply(nucleus_segm_img, dapi_orig_img)[minr:maxr, minc:maxc]))
axes[1].title.set_text('dapi nucleus')
axes[2].imshow((np.multiply(nucleus_segm_img, cebp_orig_img)[minr:maxr, minc:maxc]))
axes[2].title.set_text('cebp nucleus')
axes[3].imshow((np.multiply(nucleus_segm_img, ppar_orig_img)[minr:maxr, minc:maxc]))
axes[3].title.set_text('ppar nucleus')
axes[4].imshow(dapi_segm_img[minr:maxr, minc:maxc])
axes[4].title.set_text('heteroc mask')
axes[5].imshow(np.multiply(dapi_segm_img, dapi_orig_img)[minr:maxr, minc:maxc])
axes[5].title.set_text('heteroc dapi')
axes[6].imshow(np.multiply(dapi_segm_img, cebp_orig_img)[minr:maxr, minc:maxc])
axes[6].title.set_text('heteroc cebp')
axes[7].imshow(np.multiply(dapi_segm_img, ppar_orig_img)[minr:maxr, minc:maxc])
axes[7].title.set_text('heteroc ppar')
axes[8].imshow(n_heteroc_mask[minr:maxr, minc:maxc])
axes[8].title.set_text('n heteroc mask')
axes[9].imshow(np.multiply(n_heteroc_mask, dapi_orig_img)[minr:maxr, minc:maxc])
axes[9].title.set_text('n heteroc dapi')
axes[10].imshow(np.multiply(n_heteroc_mask, cebp_orig_img)[minr:maxr, minc:maxc])
axes[10].title.set_text('n heteroc cebp')
axes[11].imshow(np.multiply(n_heteroc_mask, ppar_orig_img)[minr:maxr, minc:maxc])
axes[11].title.set_text('n heteroc ppar')
axes[12].imshow(p3_heteroc_mask[minr:maxr, minc:maxc])
axes[12].title.set_text('p3 heteroc mask')
axes[13].imshow(np.multiply(p3_heteroc_mask, dapi_orig_img)[minr:maxr, minc:maxc])
axes[13].title.set_text('p3 heteroc dapi')
axes[14].imshow(np.multiply(p3_heteroc_mask, cebp_orig_img)[minr:maxr, minc:maxc])
axes[14].title.set_text('p3 heteroc cebp')
axes[15].imshow(np.multiply(p3_heteroc_mask, ppar_orig_img)[minr:maxr, minc:maxc])
axes[15].title.set_text('p3 heteroc ppar')
axes[16].imshow(n_p3_heteroc_mask[minr:maxr, minc:maxc])
axes[16].title.set_text('n p3 heteroc mask')
axes[17].imshow(np.multiply(n_p3_heteroc_mask, dapi_orig_img)[minr:maxr, minc:maxc])
axes[17].title.set_text('n p3 heteroc dapi')
axes[18].imshow(np.multiply(n_p3_heteroc_mask, cebp_orig_img)[minr:maxr, minc:maxc])
axes[18].title.set_text('n p3 heteroc cebp')
axes[19].imshow(np.multiply(n_p3_heteroc_mask, ppar_orig_img)[minr:maxr, minc:maxc])
axes[19].title.set_text('n p3 heteroc ppar')


 

In [125]:
fig, axes = plt.subplots(4,10)
axes = axes.flatten()
axes[0].imshow(np.array(nucleus_segm_img)[minr:maxr, minc:maxc])
axes[0].title.set_text('segmented nucleus')
axes[1].imshow((np.multiply(nucleus_segm_img, dapi_orig_img)[minr:maxr, minc:maxc]))
axes[1].title.set_text('dapi nucleus')
axes[2].imshow((np.multiply(nucleus_segm_img, cebp_orig_img)[minr:maxr, minc:maxc]))
axes[2].title.set_text('cebp nucleus')
axes[3].imshow((np.multiply(nucleus_segm_img, ppar_orig_img)[minr:maxr, minc:maxc]))
axes[3].title.set_text('ppar nucleus')
axes[4].imshow(cebp_segm_img[minr:maxr, minc:maxc])
axes[4].title.set_text('blob mask')
axes[5].imshow(np.multiply(cebp_segm_img, dapi_orig_img)[minr:maxr, minc:maxc])
axes[5].title.set_text('blob dapi')
axes[6].imshow(np.multiply(cebp_segm_img, cebp_orig_img)[minr:maxr, minc:maxc])
axes[6].title.set_text('blob cebp')
axes[7].imshow(np.multiply(cebp_segm_img, ppar_orig_img)[minr:maxr, minc:maxc])
axes[7].title.set_text('blob ppar')
axes[8].imshow(n_blob_mask[minr:maxr, minc:maxc])
axes[8].title.set_text('n blob mask')
axes[9].imshow(np.multiply(n_blob_mask, dapi_orig_img)[minr:maxr, minc:maxc])
axes[9].title.set_text('n blob dapi')
axes[10].imshow(np.multiply(n_blob_mask, cebp_orig_img)[minr:maxr, minc:maxc])
axes[10].title.set_text('n blob cebp')
axes[11].imshow(np.multiply(n_blob_mask, ppar_orig_img)[minr:maxr, minc:maxc])
axes[11].title.set_text('n blob ppar')
axes[12].imshow(p1_blob_mask[minr:maxr, minc:maxc])
axes[12].title.set_text('p1 blob mask')
axes[13].imshow(np.multiply(p1_blob_mask, dapi_orig_img)[minr:maxr, minc:maxc])
axes[13].title.set_text('p1 blob dapi')
axes[14].imshow(np.multiply(p1_blob_mask, cebp_orig_img)[minr:maxr, minc:maxc])
axes[14].title.set_text('p1 blob cebp')
axes[15].imshow(np.multiply(p1_blob_mask, ppar_orig_img)[minr:maxr, minc:maxc])
axes[15].title.set_text('p1 blob ppar')
axes[16].imshow(n_p1_blob_mask[minr:maxr, minc:maxc])
axes[16].title.set_text('n p1 blob mask')
axes[17].imshow(np.multiply(n_p1_blob_mask, dapi_orig_img)[minr:maxr, minc:maxc])
axes[17].title.set_text('n p1 blob dapi')
axes[18].imshow(np.multiply(n_p1_blob_mask, cebp_orig_img)[minr:maxr, minc:maxc])
axes[18].title.set_text('n p1 blob cebp')
axes[19].imshow(np.multiply(n_p1_blob_mask, ppar_orig_img)[minr:maxr, minc:maxc])
axes[19].title.set_text('n p1 blob ppar')
axes[20].imshow(p2_blob_mask[minr:maxr, minc:maxc])
axes[20].title.set_text('p2 blob mask')
axes[21].imshow(np.multiply(p2_blob_mask, dapi_orig_img)[minr:maxr, minc:maxc])
axes[21].title.set_text('p2 blob dapi')
axes[22].imshow(np.multiply(p2_blob_mask, cebp_orig_img)[minr:maxr, minc:maxc])
axes[22].title.set_text('p2 blob cebp')
axes[23].imshow(np.multiply(p2_blob_mask, ppar_orig_img)[minr:maxr, minc:maxc])
axes[23].title.set_text('p2 blob ppar')
axes[24].imshow(n_p2_blob_mask[minr:maxr, minc:maxc]) 
axes[24].title.set_text('n p2 blob mask')
axes[25].imshow(np.multiply(n_p2_blob_mask, dapi_orig_img)[minr:maxr, minc:maxc])
axes[25].title.set_text('n p2 blob dapi')
axes[26].imshow(np.multiply(n_p2_blob_mask, cebp_orig_img)[minr:maxr, minc:maxc])
axes[26].title.set_text('n p2 blob cebp')
axes[27].imshow(np.multiply(n_p2_blob_mask, ppar_orig_img)[minr:maxr, minc:maxc])
axes[27].title.set_text('n p2 blob ppar')
axes[28].imshow(p3_blob_mask[minr:maxr, minc:maxc])
axes[28].title.set_text('p3 blob mask')
axes[29].imshow(np.multiply(p3_blob_mask, dapi_orig_img)[minr:maxr, minc:maxc])
axes[29].title.set_text('p3 blob dapi')
axes[30].imshow(np.multiply(p3_blob_mask, cebp_orig_img)[minr:maxr, minc:maxc])
axes[30].title.set_text('p3 blob cebp')
axes[31].imshow(np.multiply(p3_blob_mask, ppar_orig_img)[minr:maxr, minc:maxc])
axes[31].title.set_text('p3 blob ppar')
axes[32].imshow(p5_blob_mask[minr:maxr, minc:maxc])
axes[32].title.set_text('p5 blob mask')
axes[33].imshow(np.multiply(p5_blob_mask, dapi_orig_img)[minr:maxr, minc:maxc])
axes[33].title.set_text('p5 blob dapi')
axes[34].imshow(np.multiply(p5_blob_mask, cebp_orig_img)[minr:maxr, minc:maxc])
axes[34].title.set_text('p5 blob cebp')
axes[35].imshow(np.multiply(p5_blob_mask, ppar_orig_img)[minr:maxr, minc:maxc])
axes[35].title.set_text('p5 blob ppar')
axes[36].imshow(p7_blob_mask[minr:maxr, minc:maxc])
axes[36].title.set_text('p7 blob mask')
axes[37].imshow(np.multiply(p7_blob_mask, dapi_orig_img)[minr:maxr, minc:maxc])
axes[37].title.set_text('p7 blob dapi')
axes[38].imshow(np.multiply(p7_blob_mask, cebp_orig_img)[minr:maxr, minc:maxc])
axes[38].title.set_text('p7 blob cebp')
axes[39].imshow(np.multiply(p5_blob_mask, ppar_orig_img)[minr:maxr, minc:maxc])
axes[39].title.set_text('p7 blob ppar')



In [123]:
fig, axes = plt.subplots(1,2)
axes = axes.flatten()

axes[0].imshow((np.multiply(nucleus_segm_img, cebp_orig_img)[minr:maxr, minc:maxc]))
axes[1].imshow((np.multiply(nucleus_segm_img, cebp_orig_img)[minr:maxr, minc:maxc]))
axes[1].imshow(np.multiply(cebp_orig_img, cebp_segm_img)[minr:maxr, minc:maxc], cmap = 'Reds', alpha = .3)


In [79]:
def invert_mask(masked_img_):
    masked_img_max = np.max(masked_img_)
    masked_img_binary = np.divide(masked_img_, masked_img_max)
    ones_bool = np.ones([masked_img_.shape[0], masked_img_.shape[1]])
    return(np.subtract(ones_bool, masked_img_binary))

In [80]:
n_cebp_masked_img = invert_mask(cebp_segm_img)

In [81]:
fig, axes = plt.subplots(1,3)
axes = axes.flatten()
axes[0].imshow(n_cebp_masked_img[minr:maxr, minc:maxc])
axes[1].imshow(nucleus_segm_img[minr:maxr, minc:maxc])
axes[2].imshow(np.multiply(nucleus_segm_img, n_cebp_masked_img)[minr:maxr, minc:maxc])

In [18]:
segm_dapi_img_filename = segm_cebp_img_filename[:-19]+'dapi'+segm_cebp_img_filename[-15:]
segm_dapi_img_filename

'stad3-26-xy1_dapi_segm_cell1.tif'

In [20]:
segm_cebp_img_filename[:-20]+'segm_parsed.tif'

'stad3-26-xy1'

In [8]:
os.chdir(cebp_segm_dir)

'''pull all of the .tif files out of the directory and put them into a list'''
cebp_segm_img_lst = []
for filename in os.listdir():
    if '.tif' in filename:
        cebp_segm_img_lst.append(filename)

        
filename_base = 'this is for a little later in code to check if files are already read into memory'

#########################################################
cebp_segm_img_lst = [cebp_segm_img_lst[10]] #this is temporary while working on code
################################################

for segm_cebp_img_filename in cebp_segm_img_lst:
    #segm_cebp_imagename_global_lst.append(segm_cebp_img_filename) #add filename to running list for bookkeeping
   
    
    cebp_segm_img = io.imread(segm_cebp_img_filename) #read in cebp single cell segmented image
    
    
    os.chdir(dapi_segm_dir) #switch over to segmented dapi directory and read in segmented dapi image'''
    segm_dapi_img_filename = segm_cebp_img_filename[:-19]+'dapi'+segm_cebp_img_filename[-15:]
    dapi_segm_img = io.imread(segm_dapi_img_filename)
    
    '''the segmented nuclei image and the fluorescent images may already be in memory, in which case can save time by just leaving them
    there and not reading them in over and over'''
    if filename_base != segm_cebp_img_filename[:-20]:
        '''read in segmented nuclei and fluorescent images'''
        filename_base = segm_cebp_img_filename[:-20]       
        os.chdir(segm_img_dir) #switch over to the segmented nuceli image directory and pull out the individual nuclei'''
        nuclei_segm_img = io.imread(segm_cebp_img_filename[:-20]+'segm_parsed.tiff') #pull out the base filename in order to identify other images associated with this mask'''

    
    '''pull out the individual nuclei from this image that corresponds to the segmented cebp and dapi images'''
    labeled_nuclei_segm_img = measure.label(nuclei_segm_img) #label the objects within the image 
    nuclei_number = int(segm_cebp_img_filename[-5]) #find the associated nucleus in the image based on the filename
    nucleus_segm_img = np.array(labeled_nuclei_segm_img == nuclei_number).astype('uint16') #pull out the single nuclues

In [20]:
plt.imshow(nucleus_segm_img)

In [21]:
measure.regionprops(nucleus_segm_img)[0].centroid[0]

1801.0302517214132

In [53]:
fig, axes= plt.subplots(2,3)
axes = axes.flatten()

axes[0].imshow(nucleus_segm_img)
axes[1].imshow(cebp_segm_img)
axes[2].imshow(dapi_segm_img)
axes[3].imshow(dapi_orig_img)
axes[4].imshow(cebp_orig_img)
axes[5].imshow(ppar_orig_img)

In [33]:
orig_img_dir_base+'\\'+segm_cebp_img_filename[:-24]

'Z:\\TAD\\190718_3t3l1_time_course_r5\\stad3-26'

In [51]:
jnk_ = binary_dilation(cebp_segm_img, disk(5))

fig, axes = plt.subplots(1,2)
axes = axes.flatten()

axes[0].imshow(cebp_segm_img)
axes[1].imshow(jnk_)

In [135]:
os.chdir(r'Z:\TAD\200428_adipogenesis_timecourses_revisited\190718_3t3l1_time_course_r5\filtered_segm_imgs')

for i in os.listdir():
    print(len(i))

28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
9
